In [1]:
import torch
import pytorch_lightning as pl
import transformers
import torchmetrics

import pandas as pd
import os
import json

from model import T5MultiTask
from data_module import TolokaDataModule

/home/stc/rybin-as/miniconda3/envs/asr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# proxy
os.environ["http_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["https_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["ftp_proxy"] = "http://proxy.ad.speechpro.com:3128"

In [3]:
t5 = transformers.T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small-chitchat2")
tokenizer = transformers.AutoTokenizer.from_pretrained("cointegrated/rut5-small-chitchat2", truncation_side='left', padding_side='right')

with open('/home/stc/persona/data/preprocessing/spec_tokens.json') as spec_tokens_config:
    spec_tokens = json.load(spec_tokens_config)
tokenizer.add_special_tokens(
            {"additional_special_tokens": [spec_tokens[k] for k in spec_tokens]}
        )

15

In [4]:
datamodule=TolokaDataModule(
    data_dir='/home/stc/persona/data',
    datasets=['current_gk'], #'next_answer', 'current_gk', 'next_gk'
    tokenizer_d=tokenizer,
    spec_tokens=spec_tokens,
    train_batch_size=64,
    val_batch_size=128,
    test_batch_size=128,
)

/home/stc/rybin-as/miniconda3/envs/asr/lib/python3.8/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [5]:
model = T5MultiTask(
    model=t5,
    tokenizer=tokenizer,
    datasets=['current_gk'],
    lr=5e-4,
    num_warmup_steps=100,
    pooling="cls",
    distance="cosine",
    scale=20,
    val_batch_size=128,
)

/home/stc/rybin-as/miniconda3/envs/asr/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(


In [6]:
# logger
logger = pl.loggers.comet.CometLogger(
    api_key='sEJsZrYjwc0gxxUAUGQNBwTsb',
    save_dir='/home/stc/persona/logs',
    project_name='T5',
    experiment_name='tripletloss',
)

CometLogger will be initialized in online mode


In [7]:
# trainer
trainer = pl.Trainer(
    max_epochs=30,
    accelerator="gpu",
    devices=1,
    gradient_clip_val=1,
    logger=logger,
    num_sanity_val_steps=10,
)
trainer.fit(model, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
Loading cached shuffled indices for dataset at /home/stc/persona/data/current_gk/train/cache-4a62906eae6ae068.arrow
100%|██████████| 2377/2377 [00:12<00:00, 184.02ba/s]

  | Name                   | Type                       | Params
----------------------------------------------------------------------
0 | transformer            | T5ForConditionalGeneration | 64.8 M
1 | cross_entropy_loss     | CrossEntropyLoss           | 0     
2 | val_current_gk_metrics | MetricCollection           | 0     
----------------------------------------------------------------------
64.8 M    Trainable params
0         Non-trainable params
64.8 M    Total params
259.049   Total estimated model params size (MB)
COMET WARNING: Comet has disabled aut

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


COMET INFO: Experiment is live on comet.com https://www.comet.com/anpopaicoconat/t5/6d7f96b047064f6ba05e7c81b1ff0d71

COMET WARNING: truncated string; too long: 'T5ForConditionalGeneration(
  (shared): Embedding(20100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(20100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
            

Sanity Checking: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Loading cached shuffled indices for dataset at /home/stc/persona/data/current_gk/train/cache-4a62906eae6ae068.arrow
Parameter 'function'=<function TolokaDataModule.train_dataloader.<locals>.<listcomp>.<lambda> at 0x7fd1c04934c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 2377/2377 [00:12<00:00, 186.80ba/s]


Epoch 29: 100%|██████████| 2636/2636 [04:49<00:00,  9.12it/s, loss=0.0229, v_num=0d71]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 2636/2636 [04:50<00:00,  9.09it/s, loss=0.0229, v_num=0d71]


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/anpopaicoconat/t5/6d7f96b047064f6ba05e7c81b1ff0d71
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     val_BLEU1 [30] : (0.16202090680599213, 0.20525911450386047)
COMET INFO:     val_BLEU2 [30] : (0.12372943758964539, 0.17951925098896027)
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:     Name         : tripletloss
COMET INFO:   Parameters:
COMET INFO:     data_dir                   : /home/stc/persona/data
COMET INFO:     datasets                   : ['current_gk']
COMET INFO:     distance                   : cosine
COMET INFO:     lr                         : 0.0005
COMET INFO:     model                      : T5ForConditionalGeneration(
  (shared): Embedding(20100, 512)
  (encoder): T5Stack(
    (embed